### Video Llava

Purpose:
- Process the `txt` files, tokenize it, convert into embeddings for input into Transformer Classification Model

Tokenizer:
- Currently using BERT tokenizer
- Each `txt` tokenize takes 5s (approx.)

### Input and Output

Input files:
- Video-llava `txt` files (in a folder)
- Label `xlsx` file (1 file)

Output files:
- `npy` file

In [1]:
import glob
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
from transformers import BertTokenizer, BertModel
import torch

#feature_files = glob.glob("EmotiW2023 Data Small/videollava/*.txt") # Video llava txt folder
file_path = 'EmotiW2023 Data Small/videollava/'
label_path = 'EmotiW2023 Data Small/engagement_labels.xlsx'  # label xlsx file here

n_segments = 10      # Number of segments per video
max_length = 512     # BERT's maximum sequence length
embedding_dim = 768  # BERT base hidden size

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained('bert-base-cased')

C:\Python39\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
C:\Python39\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [8]:
file_path = 'EmotiW2023 Data Small/videollava/'

def get_features(text_segment):
    """
    Convert text segment to fixed-size BERT embeddings
    Returns: numpy array of shape (embedding_dim,)
    """
    if not text_segment or text_segment.isspace():
        return np.zeros(embedding_dim)
    
    # Tokenize and encode text
    inputs = tokenizer(text_segment, 
                      return_tensors="pt",
                      max_length=max_length,
                      padding='max_length',
                      truncation=True)
    
    # Get BERT embeddings
    with torch.no_grad():
        outputs = model(**inputs)
        # Use CLS token embedding (first token)
        embeddings = outputs.last_hidden_state[0][0]
    
    return embeddings.numpy()

def parse_features(file_path):
    """
    Read text file and extract fixed-size features for segments
    Returns: numpy array of shape (n_segments, embedding_dim)
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read()
        
        # Split text into n_segments
        words = text.split()
        segment_size = max(1, len(words) // n_segments)
        
        # Initialize fixed-size array for features
        instance_features = np.zeros((n_segments, embedding_dim))
        
        for i in range(n_segments):
            start_idx = i * segment_size
            end_idx = start_idx + segment_size if i < n_segments - 1 else len(words)
            segment_text = ' '.join(words[start_idx:end_idx])
            
            features = get_features(segment_text)
            instance_features[i] = features
        
        return instance_features
    
    except Exception as e:
        print(f"Error processing {file_path}: {str(e)}")
        return np.zeros((n_segments, embedding_dim))

In [9]:
def path_to_txt(fname):
    return fname.split('.mp4')[0] + '.txt'

_errors_ = []
def extract_text_features():
    labels = pd.read_excel(label_path)
    data = []
    
    for f in tqdm(labels.to_dict(orient='records')):
        try:
            fname = path_to_txt(f['chunk'])
            fname = file_path + fname.split('/')[-1]
            
            features = parse_features(fname)
            # Verify shape before adding to data
            assert features.shape == (n_segments, embedding_dim), f"Wrong shape: {features.shape}"
            data.append((fname, features, f['label']))
                
        except FileNotFoundError:
            _errors_.append(fname)
        except Exception as e:
            print(f"Exception: {e}")
            _errors_.append(fname)
    
    return np.array(data, dtype='object')



In [10]:
# Extract features
Xy = extract_text_features()

# Verify shapes before saving
for item in Xy:
    assert item[1].shape == (n_segments, embedding_dim), f"Wrong shape in {item[0]}: {item[1].shape}"

Xy[0]

  6%|████▉                                                                        | 515/8040 [00:00<00:01, 5118.29it/s]

Error processing EmotiW2023 Data Small/videollava/subject_68_0ng3yqwrg6_vid_0_0.txt: [Errno 2] No such file or directory: 'EmotiW2023 Data Small/videollava/subject_68_0ng3yqwrg6_vid_0_0.txt'
Error processing EmotiW2023 Data Small/videollava/subject_68_0ng3yqwrg6_vid_0_1.txt: [Errno 2] No such file or directory: 'EmotiW2023 Data Small/videollava/subject_68_0ng3yqwrg6_vid_0_1.txt'
Error processing EmotiW2023 Data Small/videollava/subject_68_0ng3yqwrg6_vid_0_10.txt: [Errno 2] No such file or directory: 'EmotiW2023 Data Small/videollava/subject_68_0ng3yqwrg6_vid_0_10.txt'
Error processing EmotiW2023 Data Small/videollava/subject_68_0ng3yqwrg6_vid_0_11.txt: [Errno 2] No such file or directory: 'EmotiW2023 Data Small/videollava/subject_68_0ng3yqwrg6_vid_0_11.txt'
Error processing EmotiW2023 Data Small/videollava/subject_68_0ng3yqwrg6_vid_0_12.txt: [Errno 2] No such file or directory: 'EmotiW2023 Data Small/videollava/subject_68_0ng3yqwrg6_vid_0_12.txt'
Error processing EmotiW2023 Data Small/

 35%|███████████████████████████▏                                                  | 2800/8040 [07:21<01:18, 66.55it/s]

Error processing EmotiW2023 Data Small/videollava/subject_108_randomn93c6rsmts_vid_0_8.txt: [Errno 2] No such file or directory: 'EmotiW2023 Data Small/videollava/subject_108_randomn93c6rsmts_vid_0_8.txt'
Error processing EmotiW2023 Data Small/videollava/subject_108_randomn93c6rsmts_vid_0_9.txt: [Errno 2] No such file or directory: 'EmotiW2023 Data Small/videollava/subject_108_randomn93c6rsmts_vid_0_9.txt'
Error processing EmotiW2023 Data Small/videollava/subject_108_randomn93c6rsmts_vid_1_0.txt: [Errno 2] No such file or directory: 'EmotiW2023 Data Small/videollava/subject_108_randomn93c6rsmts_vid_1_0.txt'
Error processing EmotiW2023 Data Small/videollava/subject_108_randomn93c6rsmts_vid_1_1.txt: [Errno 2] No such file or directory: 'EmotiW2023 Data Small/videollava/subject_108_randomn93c6rsmts_vid_1_1.txt'
Error processing EmotiW2023 Data Small/videollava/subject_108_randomn93c6rsmts_vid_1_10.txt: [Errno 2] No such file or directory: 'EmotiW2023 Data Small/videollava/subject_108_rand

 38%|█████████████████████████████▌                                                | 3050/8040 [07:50<03:09, 26.31it/s]

...

 89%|████████████████████████████████████████████████████████████████████▎        | 7134/8040 [09:21<00:02, 336.14it/s]

Error processing EmotiW2023 Data Small/videollava/subject_15_randomdradnfzte6_vid_0_10.txt: [Errno 2] No such file or directory: 'EmotiW2023 Data Small/videollava/subject_15_randomdradnfzte6_vid_0_10.txt'
Error processing EmotiW2023 Data Small/videollava/subject_15_randomdradnfzte6_vid_0_11.txt: [Errno 2] No such file or directory: 'EmotiW2023 Data Small/videollava/subject_15_randomdradnfzte6_vid_0_11.txt'
Error processing EmotiW2023 Data Small/videollava/subject_15_randomdradnfzte6_vid_0_12.txt: [Errno 2] No such file or directory: 'EmotiW2023 Data Small/videollava/subject_15_randomdradnfzte6_vid_0_12.txt'
Error processing EmotiW2023 Data Small/videollava/subject_15_randomdradnfzte6_vid_0_13.txt: [Errno 2] No such file or directory: 'EmotiW2023 Data Small/videollava/subject_15_randomdradnfzte6_vid_0_13.txt'
Error processing EmotiW2023 Data Small/videollava/subject_15_randomdradnfzte6_vid_0_14.txt: [Errno 2] No such file or directory: 'EmotiW2023 Data Small/videollava/subject_15_random

100%|██████████████████████████████████████████████████████████████████████████████| 8040/8040 [09:21<00:00, 14.32it/s]

Error processing EmotiW2023 Data Small/videollava/subject_67_ra9y2mtmgw_vid_1_30.txt: [Errno 2] No such file or directory: 'EmotiW2023 Data Small/videollava/subject_67_ra9y2mtmgw_vid_1_30.txt'
Error processing EmotiW2023 Data Small/videollava/subject_67_ra9y2mtmgw_vid_1_31.txt: [Errno 2] No such file or directory: 'EmotiW2023 Data Small/videollava/subject_67_ra9y2mtmgw_vid_1_31.txt'
Error processing EmotiW2023 Data Small/videollava/subject_67_ra9y2mtmgw_vid_1_32.txt: [Errno 2] No such file or directory: 'EmotiW2023 Data Small/videollava/subject_67_ra9y2mtmgw_vid_1_32.txt'
Error processing EmotiW2023 Data Small/videollava/subject_67_ra9y2mtmgw_vid_1_33.txt: [Errno 2] No such file or directory: 'EmotiW2023 Data Small/videollava/subject_67_ra9y2mtmgw_vid_1_33.txt'
Error processing EmotiW2023 Data Small/videollava/subject_67_ra9y2mtmgw_vid_1_34.txt: [Errno 2] No such file or directory: 'EmotiW2023 Data Small/videollava/subject_67_ra9y2mtmgw_vid_1_34.txt'
Error processing EmotiW2023 Data Sm

array(['EmotiW2023 Data Small/videollava/subject_68_0ng3yqwrg6_vid_0_0.txt',
       array([[0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.],
              ...,
              [0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.]]), 'Engaged'], dtype=object)

In [38]:
np.save('EmotiW2023 Data Small/Xy_engage_llava.npy', Xy)

In [39]:
def path_to_mp4(fname):
    return fname.split('.txt')[0] + '.mp4'

Xy_pdm = np.load('EmotiW2023 Data Small/Xy_engage_llava.npy', allow_pickle=True)

In [40]:
for i in range(len(Xy_pdm)):
    #Xy_pdm[i][0] = 'chunks/' + path_to_mp4(Xy_pdm[i][0].split('/')[-1])
    Xy_pdm[i][0] = path_to_mp4(Xy_pdm[i][0].split('/')[-1])

Xy_pdm[0]

array(['subject_68_0ng3yqwrg6_vid_0_0.mp4',
       array([[0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.],
              ...,
              [0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.],
              [0., 0., 0., ..., 0., 0., 0.]]), 'Engaged'], dtype=object)

In [41]:
np.save('EmotiW2023 Data Small/Xy_engage_llava.npy', Xy_pdm)

### Below is Testing

Just making sure everything works

In [42]:
index = 200
print(Xy_pdm[index][0])
print(Xy_pdm[index][1][0])

subject_2_thw33jke4j_vid_2_39.mp4
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0

In [43]:
target_filename = 'subject_0_2msdhgqawh_vid_0_0.mp4'

for i in range(len(Xy_pdm)):
    if Xy_pdm[i][0] == target_filename:
        print(f"Found at index: {i}")
        print("Filename:", Xy_pdm[i][0])
        print("First row of features:", Xy_pdm[i][1][0])
        break

Found at index: 3047
Filename: subject_0_2msdhgqawh_vid_0_0.mp4
First row of features: [-2.57278353e-01  1.77065387e-01 -1.39969708e-02 -5.30950353e-02
 -1.17407907e-02 -7.58732855e-02  9.27412435e-02 -2.95427144e-02
  2.16143295e-01 -1.37159872e+00 -2.76598990e-01  1.45899162e-01
 -4.31050807e-01  2.62471914e-01 -7.53263831e-01  2.38295034e-01
  1.58585548e-01 -2.16551840e-01 -3.97073209e-01 -4.75823253e-01
  1.74612328e-01 -1.19449720e-01  4.34893250e-01 -5.13995588e-01
  2.92093128e-01 -1.80640250e-01  2.13761464e-01  6.88903630e-02
  2.23711133e-04  3.40212226e-01 -6.45334795e-02  1.06989361e-01
 -1.16965964e-01 -1.63618982e-01  2.02538669e-01  2.14019939e-02
  2.56913394e-01 -3.63281399e-01  4.43476439e-02  3.80553640e-02
 -4.50329453e-01 -4.09023799e-02  8.79179060e-01  1.84856892e-01
 -4.80119288e-02 -5.68004251e-01  2.67740250e-01  2.59410143e-01
 -1.60430878e-01  2.12724954e-01  7.48054460e-02  1.49399579e-01
 -1.21759124e-01  8.49147812e-02  1.52021702e-02  9.49265212e-02
 -3